# OpenAI Completions Workshop Notebook

End-to-end exploration of OpenAI *text completion* patterns: listing models, baseline prompting, refinement, multiple candidates, token awareness, penalties, and streaming.

Environment requirement: `OPENAI_API_KEY` must be set (or a `.env` loaded earlier). Optional: set `OPENAI_MODEL` to override the default model.

In [14]:
# Imports & client initialization
import os
import time
import math
from openai import OpenAI

API_KEY = os.getenv('OPENAI_API_KEY')
if not API_KEY:
    raise EnvironmentError(
        'OPENAI_API_KEY not set. Set it in your environment or .env file.')

# Override by exporting OPENAI_MODEL if desired
MODEL = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
client = OpenAI(api_key=API_KEY)
print(f'Using model: {MODEL}')

Using model: gpt-4o-mini


## 1. List Available Models
Shows the raw list returned by the Models API (truncated for brevity).

In [ ]:
models = client.models.list()
print(f'Total models: {len(models.data)}')
for m in models.data[:10]:  # show first 10
    print(f'{m.id:40} | owned_by={getattr(m, 'owned_by', '?')}')

## 2. Baseline Completion
Single prompt -> single completion.

In [ ]:
prompt = (
    'Suggest three concise brand name ideas for an eco-friendly smart home cleaning device. '
    'Each name should feel modern, memorable, and avoid generic buzzwords.'
)
response = client.completions.create(
    model=MODEL, prompt=prompt, max_tokens=120, temperature=0.7)
print(response.choices[0].text.strip())

## 3. Multiple Candidates & Temperature Sweep
Generate several candidates to compare diversity.

In [ ]:
temps = [0.2, 0.7, 1.0]
for t in temps:
    resp = client.completions.create(
        model=MODEL, prompt=prompt, max_tokens=100, n=3, temperature=t)
    print(f'--- temperature={t} ---')
    for i, choice in enumerate(resp.choices, 1):
        print(f'[{i}] {choice.text.strip()}')
    print()

## 4. Presence & Frequency Penalties
Encourage novelty or reduce repetition. (Values here are illustrative; tune per task.)

In [ ]:
for presence in [0.0, 0.8]:
    for freq in [0.0, 0.8]:
        resp = client.completions.create(
            model=MODEL, prompt=prompt, max_tokens=80, temperature=0.7,
            presence_penalty=presence, frequency_penalty=freq, n=1)
        print(f'presence_penalty={presence} frequency_penalty={freq}')
        print(resp.choices[0].text.strip(), '---')

## 5. Token Counting (tiktoken)
Estimate prompt token usage for budgeting / compression.

In [ ]:
try:
    import tiktoken
    enc = tiktoken.get_encoding('cl100k_base')
    token_count = len(enc.encode(prompt))
    print('Prompt tokens:', token_count)
except Exception as e:
    print('tiktoken not installed or error occurred:', e)

## 6. Streaming Responses
Stream tokens as they arrive for a longer generation.

In [ ]:
long_prompt = (
    'Generate three distinct premium brand names and for each a two-sentence tagline for '
    'an eco-friendly smart home cleaning device brand. Emphasize sustainability, smart automation, and ease of use.'
)
print('Streaming start ->')
for chunk in client.completions.create(model=MODEL, prompt=long_prompt, max_tokens=200, temperature=0.8, stream=True):
    for c in chunk.choices:
        text = getattr(c, 'text', '')
        if text:
            print(text, end='')
print('\n\n<- Streaming end')

## 7. Simple Helper Wrapper with Retry
Basic utility to standardize calls & add retry-on-failure.

In [ ]:
import random
from typing import List
from openai import APIError, RateLimitError


def generate(prompt: str, *, model: str = MODEL, n: int = 1, temperature: float = 0.7, max_tokens: int = 100, retries: int = 3, backoff: float = 1.5) -> List[str]:
    attempt = 0
    while True:
        try:
            resp = client.completions.create(
                model=model, prompt=prompt, n=n, temperature=temperature, max_tokens=max_tokens)
            return [c.text.strip() for c in resp.choices]
        except (RateLimitError, APIError) as e:
            attempt += 1
            if attempt > retries:
                raise
            sleep_for = backoff ** attempt + random.random()
            print(
                f'Retry {attempt}/{retries} after error: {e}. Sleeping {sleep_for:.2f}s')
            time.sleep(sleep_for)


# New neutral sample prompt
sample_prompt = 'List two differentiating value propositions for an eco-friendly smart home cleaning device.'
results = generate(sample_prompt, n=2, temperature=0.6)
for i, r in enumerate(results, 1):
    print(f'Choice {i}: {r}')

## 8. Recap & Next Steps
You explored: model listing, baseline prompting, multiple candidates, penalties, token counting, streaming, and retries.
Next ideas: add evaluation, cost estimation, structured extraction with stop sequences or JSON mode (if supported).